# Design patterns

Design patterns are template solutions you can use for recurring problems in software design. They are independent of programming languages, although some languages may make their implementation easier. Because of their very nature, design patterns are very abstract and realizing that a problem lends itself to a particular design pattern can be challenging. Using design patterns, however, allows adopting a common "design language" that can be easily understood by other programmers familiar with the pattern looking at your code.

Design patterns have a long history, but the 1994 book _"Design Patterns: Elements of Reusable Object-Oriented Software"_ by Erich Gamma, Richard Helm, Ralph Johnson, and John Vlissides (known as the Gang of Four) largely influenced how people think about them today.

The book defines three types of patterns:

### Creational

- Patterns dealing with the creation of objects beyond simple instantiation.

### Structural

- Patterns concerning class composition to achieve specific functionality.

### Behavioral

- Patterns concerned with the communication or interaction between classes.

<div class="alert alert-block alert-info">

**Note:** Design patterns are not without criticism. Some people consider them mere workarounds for missing features in a programming language. While this may be true in some cases, it does not mean they are not useful, and knowing them can help to understand advanced language features more easily.

</div>

The Gang of Four lists 23 patterns in their book. Since we cannot cover all of them in this notebook, we will focus one one from each type. You can check out the rest of them [here](https://en.wikipedia.org/wiki/Design_Patterns#Patterns_by_type).


## Example creational pattern: Factory

When we need to create an object, we usually simply instantiate it directly. There are circumstances, however, where that may become cumbersome. For example, let's say we are writing a game and need to spawn monsters in an area. We have created a couple of different `Monster` subclasses:


In [ ]:
class Monster:
    def __init__(self, health):
        self.health = health

    def attack(self):
        pass

    def defend(self):
        pass

    def special(self):
        pass


class Orc(Monster):
    def __init__(
        self,
        health=5,
    ):
        super().__init__(health)

    def attack(self):
        print("The orc attacks!")

    def defend(self):
        print("The orc parries!")

    def special(self):
        print("The orc lets out a war cry!")


class Kobold(Monster):
    def __init__(self, health=3):
        super().__init__(health)

    def attack(self):
        print("The kobold attacks!")

    def defend(self):
        print("The kobold dodges!")

    def special(self):
        print("The kobold sets off a trap!")


class Dragon(Monster):
    def __init__(self, health=20):
        super().__init__(health)

    def attack(self):
        print("The dragon attacks!")

    def defend(self):
        print("The dragon blocks!")

    def special(self):
        print("The dragon breathes fire!")

We could then define the monsters in an area in a list and then spawn them like so:


In [ ]:
monsters_to_spawn = ["orc", "orc", "orc", "kobold", "kobold", "dragon"]


monsters = list()
for monster in monsters_to_spawn:
    if monster == "orc":
        monsters.append(Orc())
    if monster == "kobold":
        monsters.append(Kobold())
    if monster == "dragon":
        monsters.append(Dragon())


for monster in monsters:
    monster.attack()
    monster.special()

That does the trick and is not terrible at first glance. However, what if we wanted to include a difficulty level? We would have to do something like this:


In [ ]:
monsters_to_spawn = ["orc", "orc", "orc", "kobold", "kobold", "dragon"]

difficulty = "hard"


monsters = list()
for monster in monsters_to_spawn:
    if monster == "orc":
        if difficulty == "easy":
            monsters.append(Orc(health=5))
        if difficulty == "hard":
            monsters.append(Orc(health=10))
    if monster == "kobold":
        if difficulty == "easy":
            monsters.append(Kobold(health=3))
        if difficulty == "hard":
            monsters.append(Kobold(health=6))
    if monster == "dragon":
        if difficulty == "easy":
            monsters.append(Dragon(health=20))
        if difficulty == "hard":
            monsters.append(Dragon(health=40))


for monster in monsters:
    monster.attack()
    monster.special()

The code is starting to get fairly verbose. It would get even worse if we added more kinds of monsters or more difficulty levels. It's also likely that we would encounter logic like the above in several parts of our program, which means we would have to make sure we keep all of those places up to date with any additions we might make!

The underlying problem here is as follows:

- The calling (or client) code wants to create a couple of objects
- These objects need to be created according to certain specifications (type of monster, difficulty level)
- The client has to know the implementation details of the desired objects (e.g., how the difficulty affects the hitpoints)

We can fix all of these issues by delegating the object construction to a separate object using the `Factory` pattern:


In [1]:
class MonsterFactory:
    def __init__(self, difficulty):
        self.difficulty = difficulty
        self.base_hitpoints = {"orc": 5, "kobold": 3, "dragon": 20}

    def spawn(self, monster):
        if self.difficulty == "easy":
            hitpoints = self.base_hitpoints[monster]
        if self.difficulty == "hard":
            hitpoints = self.base_hitpoints[monster] * 2

        if monster == "orc":
            return Orc(health=hitpoints)
        if monster == "kobold":
            return Kobold(health=hitpoints)
        if monster == "dragon":
            return Dragon(health=hitpoints)

Now we can use the `MonsterFactory` to spawn our monsters:


In [ ]:
monsters_to_spawn = ["orc", "orc", "orc", "kobold", "kobold", "dragon"]

monster_factory = MonsterFactory("hard")

monsters = [monster_factory.spawn(monster) for monster in monsters_to_spawn]

for monster in monsters:
    monster.attack()
    monster.special()

The client code now no longer needs to worry about the implementation of the various monster types and how the difficulty affects their stats! This is all handled in the `MonsterFactory`, which can be easily extended or adapted as we continue adding monster or balancing the difficulty.

By the way: You can find plenty of examples for the `Factory` pattern in Python. The construction of a `date` object from a timestamp, for example, uses a factory method:


In [ ]:
from datetime import date
import time

date.fromtimestamp(time.time())

Another example is the construction of a Pandas `DataFrame` from a dictionary:


In [ ]:
import pandas as pd

data = {"Numbers": [3, 2, 1, 0], "Letters": ["a", "b", "c", "d"]}
pd.DataFrame.from_dict(data)

## Example structural pattern: Decorator

Sometimes we want to add, remove, or alter functionality to an object at runtime. Consider the following example of a class for some iterative calculations:


In [ ]:
from time import sleep


class NumberCruncher:
    def __init__(self, n_iter):
        self.n_iter = n_iter

    def crunch(self):
        for _ in range(self.n_iter):
            self.do_iteration()
        print("\n")

    def do_iteration(self):
        print(".", end="")
        sleep(0.1)


nc = NumberCruncher(10)

nc.crunch()

Now what if we wanted to add some logging output to this class without changing its implementation?

We could use the `Decorator` pattern:


In [ ]:
class LoggingDecorator:
    def __init__(self, number_cruncher):
        self.number_cruncher = number_cruncher

    def crunch(self):
        print(10 * "*")
        print("Crunching...")
        self.number_cruncher.crunch()
        print("Done!")
        print(10 * "*")


nc = LoggingDecorator(NumberCruncher(10))

nc.crunch()

The `Decorator` wraps around the object we want to decorate and then copies the wrapped object's interface, adding the desired functionality around it.


## Example behavioral pattern: Memento

The `Memento` pattern can be used whenever you need to save and restore a previous state of an object. A great example is an _undo_ functionality. Assume we are implementing a simple _Guess the Word_ game:


In [ ]:
class GuessTheWord:
    def __init__(self, solution):
        self.solution = solution
        self.current = len(self.solution) * ["_"]
        self.is_solved = False

    def get_input(self):
        return input()

    def print(self):
        print(self.current)

    def run(self):
        print("Welcome to Guess The Word!")

        while not self.is_solved:
            self.print()
            s = self.get_input()
            if len(s) == 1:
                self.try_letter(s)
            else:
                self.try_solution(s)
            if self.is_solved:
                self.print()
                print("You won!")

    def try_letter(self, s):
        for idx, c in enumerate(self.solution):
            if c.lower() == s.lower():
                self.current[idx] = c
        if "_" not in self.current:
            self.is_solved = True

    def try_solution(self, s):
        if self.solution == s:
            self.current = [c for c in self.solution]
            self.is_solved = True
        else:
            print("That's not it!")


GuessTheWord("Mystery").run()

Let's say we want to add an _undo_ command to the game that moves the game back to the previous state. To do that, we first need to figure out what exactly constitutes the _state_ of the game at any point in time. In this case, the state is simply the field `current`. To implement the `Memento` pattern, we will now change the code slightly, so that before the state is changed, a snapshot of it is saved to a history:


In [ ]:
class GuessTheWord:
    def __init__(self, solution):
        self.solution = solution
        self.history = []  # We start with an empty history
        self.current = len(self.solution) * ["_"]
        self.is_solved = False

    def get_input(self):
        return input()

    def print(self):
        print(self.current)

    def run(self):
        print("Welcome to Guess The Word!")

        while not self.is_solved:
            self.print()
            s = self.get_input()
            if len(s) == 1:
                self.try_letter(s)
            elif s == "undo":
                # Restore the last state from the history
                if self.history:
                    self.current = self.history.pop()
            else:
                self.try_solution(s)

            if self.is_solved:
                self.print()
                print("You won!")

    def try_letter(self, s):
        # Add a snapshot of the current state to the history
        self.history.append(self.current.copy())

        for idx, c in enumerate(self.solution):
            if c.lower() == s.lower():
                self.current[idx] = c
        if "_" not in self.current:
            self.is_solved = True

    def try_solution(self, s):
        if self.solution == s:
            self.current = [c for c in self.solution]
            self.is_solved = True
        else:
            print("That's not it!")


GuessTheWord("Mystery").run()

<table >
<tbody>
  <tr>
    <td style="padding:0px;border-width:0px;vertical-align:center">    
    Created by Simon Stone for Dartmouth College Library under <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons CC BY-NC 4.0 License</a>.<br>For questions, comments, or improvements, email <a href="mailto:researchdatahelp@groups.dartmouth.edu">Research Data Services</a>.
    </td>
    <td style="padding:0 0 0 1em;border-width:0px;vertical-align:center"><img alt="Creative Commons License" src="https://i.creativecommons.org/l/by/4.0/88x31.png"/></td>
  </tr>
</tbody>
</table>
